In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# lifted from
# https://github.com/open-telemetry/opentelemetry-python/blob/master/ext/opentelemetry-ext-jaeger/examples/jaeger_exporter_example.py

In [3]:
from opentelemetry.sdk.trace import Tracer
from opentelemetry import trace as trace_api

try:
    trace_api.set_preferred_tracer_implementation(lambda T: Tracer())
    tracer = trace_api.tracer()
except RuntimeError:
    pass

tracer

In [4]:
import time
from opentelemetry.ext import jaeger
from opentelemetry.sdk.trace.export import BatchExportSpanProcessor
from opentelemetry.sdk.trace.export import SimpleExportSpanProcessor
from opentelemetry.sdk.trace.export import ConsoleSpanExporter

logging_sp = SimpleExportSpanProcessor(
    ConsoleSpanExporter()
)

je1 = jaeger.JaegerSpanExporter(
    service_name="Upstream",
    agent_host_name="localhost",
    agent_port=6831,
)
sp1 = BatchExportSpanProcessor(je1)
tracer.add_span_processor(sp1)
tracer.add_span_processor(logging_sp)


je2 = jaeger.JaegerSpanExporter(
    service_name="This Service",
    agent_host_name="localhost",
    agent_port=6831,
)
sp2 = BatchExportSpanProcessor(je2)
t2 = Tracer()
t2.add_span_processor(sp2)
t2.add_span_processor(logging_sp)


je3 = jaeger.JaegerSpanExporter(
    service_name="Downstream",
    agent_host_name="localhost",
    agent_port=6831,
)
sp3 = BatchExportSpanProcessor(je3)
t3 = Tracer()
t3.add_span_processor(sp3)
t3.add_span_processor(logging_sp)

In [5]:
import asyncio

async def nop(n):
    with t3.start_as_current_span("downstream {}".format(n)):
        await asyncio.sleep(.5)
        return n

with tracer.start_as_current_span("upstream") as upstream:
    await asyncio.sleep(.2)

    with t2.start_as_current_span("this one") as this_one:
        with t2.start_as_current_span("some work") as some_work:
            await asyncio.sleep(.2)

        tasks = []
        tasks.append(asyncio.create_task(nop(1)))
        await asyncio.sleep(.1)
        tasks.append(asyncio.create_task(nop(2)))
        await asyncio.sleep(.1)
        tasks.append(asyncio.create_task(nop(3)))

        for t in tasks:
            await(t)

        await asyncio.sleep(.2)
    await asyncio.sleep(.2)

Span(name="some work", context=SpanContext(trace_id=0x8714f6b0dd95e9d704e55ce9068e946f, span_id=0x3e10bb72e78a4e5c, trace_state={}), kind=SpanKind.INTERNAL, parent=Span(name="this one", context=SpanContext(trace_id=0x8714f6b0dd95e9d704e55ce9068e946f, span_id=0xa075b53b0b07430d, trace_state={})), start_time=2019-11-12T23:12:53.869922Z, end_time=2019-11-12T23:12:54.074509Z)
Span(name="downstream 1", context=SpanContext(trace_id=0x8714f6b0dd95e9d704e55ce9068e946f, span_id=0xee66ff917896b94f, trace_state={}), kind=SpanKind.INTERNAL, parent=Span(name="this one", context=SpanContext(trace_id=0x8714f6b0dd95e9d704e55ce9068e946f, span_id=0xa075b53b0b07430d, trace_state={})), start_time=2019-11-12T23:12:54.075153Z, end_time=2019-11-12T23:12:54.577829Z)
Span(name="downstream 2", context=SpanContext(trace_id=0x8714f6b0dd95e9d704e55ce9068e946f, span_id=0xdac51c6459e2eba0, trace_state={}), kind=SpanKind.INTERNAL, parent=Span(name="this one", context=SpanContext(trace_id=0x8714f6b0dd95e9d704e55ce9068

In [6]:
sp1.shutdown()
sp2.shutdown()
sp3.shutdown()

In [5]:
give_kubecon_talk = lambda: None

# instrumenting application code

from opentelemetry import trace

# tracer = trace.tracer() # shared tracer

with tracer.start_as_current_span("kubecon_talk") as span:
    give_kubecon_talk()
    span.add_event("applause", {"racuous": True})

Span(name="kubecon_talk", context=SpanContext(trace_id=0x4a70275724c553a28d3fee9a65231c44, span_id=0x9b77d07b464ba8f1, trace_state={}), kind=SpanKind.INTERNAL, parent=None, start_time=2019-11-13T00:13:16.025103Z, end_time=2019-11-13T00:13:16.025140Z)


In [9]:
span.events[0].__dict__

{'_name': 'applause',
 '_attributes': {'racuous': True},
 '_timestamp': 1573603996025122000}